In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import os 
from os import listdir
import numpy as np
import shutil
import glob

Dataset:LeafSnap • 23147 Lab images, consisting of high-quality images taken of pressed leaves, from the Smithsonian collection. These images appear in controlled backlit and front-lit versions, with several samples per species

My dataset: i will select 12 diffrent classes of tree species Images in field version

STEPS:
- chose 12 folders with segmented lab images
- delete useless images
- create train set(labels included, maybe np array of pixel values of image). 80% images of each class
- test set 20% images of each class
- get keypoints 
- dont know how to fit training in this problem, do i need to create neutral network, or algorithm like linear regression?
- I image this like training some masks to recognize each class, and then each element of test set will be checked witch all masks, and that one with biggest % of simmilarity will be labeled the same as mask which gave biggest %

In [3]:
part_dataset = "part_dataset"
for specie_folder in os.listdir(part_dataset):
    leaf_class_path = part_dataset + "/" + specie_folder
    class_size = len(os.listdir(leaf_class_path))
    size_080 = round(class_size * 0.8)

In [40]:
part_dataset = "part_dataset_col"
# go throught each folder
# in folder select (best randomly) 80% of files, and save them to train directory in folder with specie_name, rest files save to test_direcotry 
for specie_folder in os.listdir(part_dataset):
    leaf_class_path = part_dataset + "/" + specie_folder
    class_size = len(os.listdir(leaf_class_path))
    size_080 = round(class_size * 0.8)
    num = 1
    for leaf_img in os.listdir(leaf_class_path):
        leaf_img_path = leaf_class_path + "/" + leaf_img

        if num < size_080:
            leaf_img_dest = "my_dataset/train" + "/" + specie_folder + "/" + leaf_img
            shutil.copy(leaf_img_path, leaf_img_dest)
        else:
            leaf_img_dest = "my_dataset/test" + "/" + leaf_img
            shutil.copy(leaf_img_path, leaf_img_dest)
        num += 1

In [4]:
train_path = "my_dataset/train"
test_path = "my_dataset/test"

In [15]:
x_train = {} #dictionary where key is name of specie, value is list of image lists
y_train = []
x_test = []

for species_folder in os.listdir(train_path):
    leaf_class_path = train_path + "/" + species_folder

    leafs_arr = []
    for leaf_img in os.listdir(leaf_class_path):
        img_path = leaf_class_path + "/" + leaf_img
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        img = cv.resize(img, (500,500))
        img = np.array(img)
        leafs_arr.append(img)
    x_train[species_folder] = leafs_arr
    
for test_img in os.listdir(test_path):
    test_img_path = test_path + "/" + test_img

    img = cv.imread(test_img_path, cv.IMREAD_GRAYSCALE)
    img = cv.resize(img, (500,500))
    img = np.array(img)
    x_test.append(img)

Now i have train set with species name and images in numeric -> key, value
and test set containing list of numeric values of pixel of images

Now it is time to create mask for each class, based on train set

First i will try diffrent aproach:
get keypoints from images, use descriptors to save image descriptors in array, get simmilarities, or use weights
match set of keyponts, descriptors using brute-force matcher

iterate over one species -> 

In [ ]:
Another idea:
    have a dictionary specie -> descriptors
    go throught test images
    find matches betweend diffrent species descriptors
    count for which one there was most matches -> that is that specie

In [16]:
orb = cv.ORB_create()
species_descriptors_dict = {}
for specie, images_list in x_train.items():
    specie_descriptors = [] # list of list desciptors
    for image in images_list:
        image = np.array(image)
        keypoints, descriptors = orb.detectAndCompute(image, None) # tuple -> list of keypoints, lisft of descriptors
        specie_descriptors.append(descriptors)
    species_descriptors_dict[specie] = specie_descriptors
print(species_descriptors_dict.keys())

dict_keys(['acer_campestre', 'acer_ginnala', 'carya_ovata', 'castanea_dentata', 'corylus_colurna', 'ficus_carica', 'fraxinus_nigra', 'ginkgo_biloba', 'gymnocladus_dioicus', 'morus_alba', 'quercus_palustris', 'tilia_europaea'])


In [17]:
def average(lst):
    return sum(lst)/len(lst)

In [18]:
matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

def find_match(test_img):
    test_img_kp, test_img_des = orb.detectAndCompute(test_img, None)
    species_average_match_dict = {}
    
    for specie, specie_list_descriptors in species_descriptors_dict.items():
        lst_average_match_imgs = []     
        # one specie has list of list descriptors
        for specie_img_descriptor in specie_list_descriptors:
            # coutning average match for each foto of specie and test img
            matches = matcher.match(specie_img_descriptor, test_img_des)
            matches_dist = []
            
            for match in matches:
                matches_dist.append(match.distance)
                
            average_match_img = average(matches_dist)   
            lst_average_match_imgs.append(average_match_img)   
 
        species_average_match_dict[specie] = average(lst_average_match_imgs)
        
    specie_recognized = find_specie_closest_mean_match(species_average_match_dict)
    print("recognized as ")
    print(specie_recognized)
    print(' ')
    cv.imshow("test", test_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    

In [19]:
def find_specie_closest_mean_match(species_match_dict):
    for specie, mean_match_distance in species_match_dict.items():
        print(specie, '->', mean_match_distance)
    specie_recognized = min(species_match_dict, key=species_match_dict.get)
    return specie_recognized

In [20]:
def find_specie_closest_match(species_match_dict):
    specie_match_distance = {}
    for specie, list_matches in species_match_dict.items():
        list_sum = []
        for matches in list_matches:
            sum_distance = 0
            for match in matches:
                sum_distance += match.distance
            list_sum.append(sum_distance)
        specie_match_distance[specie] = sum(list_sum)
    for specie, match_distance in specie_match_distance.items():
        print(specie, '->', match_distance)
    specie_recognized = min(specie_match_distance, key=specie_match_distance.get)
    return specie_recognized

In [21]:
for test_image in x_test:
    find_match(test_image)

acer_campestre -> 51.95571610642553
acer_ginnala -> 51.22910590044191
carya_ovata -> 53.82797274512892
castanea_dentata -> 53.113309694859836
corylus_colurna -> 50.728399426155484
ficus_carica -> 52.21605980723829
fraxinus_nigra -> 54.75472538345939
ginkgo_biloba -> 47.32721274336848
gymnocladus_dioicus -> 56.29565379628681
morus_alba -> 53.32737593206747
quercus_palustris -> 54.94616428711452
tilia_europaea -> 56.112072695593525
recognized as 
ginkgo_biloba
 
acer_campestre -> 49.789132642517494
acer_ginnala -> 48.45054970154401
carya_ovata -> 52.7001566223157
castanea_dentata -> 49.69771495072644
corylus_colurna -> 48.72503352147622
ficus_carica -> 50.00419624375774
fraxinus_nigra -> 52.20136759368618
ginkgo_biloba -> 45.63358477074356
gymnocladus_dioicus -> 55.40409526560199
morus_alba -> 49.93476510737049
quercus_palustris -> 51.362350918289856
tilia_europaea -> 51.43963071523233
recognized as 
ginkgo_biloba
 
acer_campestre -> 48.42518359402716
acer_ginnala -> 46.83694000382476
ca

KeyboardInterrupt: 

In [12]:
orb = cv.ORB_create()
img = cv.imread("my_dataset/train/fraxinus_nigra/13291804050219.jpg", cv.IMREAD_GRAYSCALE)
img_kp, img_des = orb.detectAndCompute(img, None)
test_img = cv.imread("my_dataset/test/1258480473_0016.jpg", cv.IMREAD_GRAYSCALE)
test_img_kp, test_img_des = orb.detectAndCompute(test_img, None)

bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

matches = bf.match(img_des, test_img_des)

matches = sorted(matches, key=lambda x:x.distance)

img_matches = cv.drawMatches(
    img, img_kp, test_img, test_img_kp, matches[:25], test_img, flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv.imshow("matches", img_matches)
cv.waitKey(0)
cv.destroyAllWindows()